In [ ]:
from ultralytics import YOLO
import json
from google.cloud import storage

In [ ]:
# Download helper function
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a file from a Cloud Storage bucket after verifying the blob exists."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    if not blob.exists(storage_client):
        raise FileNotFoundError(f"Error: Blob '{source_blob_name}' not found in bucket '{bucket_name}'.")

    try:
        blob.download_to_filename(destination_file_name)
        gs_path = f"gs://{bucket_name}/{source_blob_name}"
        print(f"Downloaded storage object from {gs_path} to local file {destination_file_name}.")
        return gs_path  # Return the gs:// path

    except Exception as e:
        print(f"Error downloading file: {e}")
        return None  # Return None if an error occurs

In [ ]:
# Download the casting image file from the bucket
bucket_name = "metal_casting_images"
source_blob_name = "raw/cast_ok_0_16.jpeg"
download_file_name = source_blob_name.split("/")[-1]
download_blob(bucket_name, source_blob_name, download_file_name)

# Download the model file from the bucket
model_file_name = "v0.pt"
download_blob("metal_casting_model", model_file_name, model_file_name)
model = YOLO(model_file_name)

# Run inference on a sample casting image
results = model.predict(download_file_name, save = True, show_conf = True)

Downloaded storage object from gs://metal_casting_images/raw/cast_ok_0_16.jpeg to local file cast_ok_0_16.jpeg.
Downloaded storage object from gs://metal_casting_model/v0.pt to local file v0.pt.

image 1/1 c:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\cast_ok_0_16.jpeg: 224x224 OK 0.93, Defect 0.07, 3.3ms
Speed: 8.9ms preprocess, 3.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Results saved to runs\classify\predict39


In [ ]:
# Print the results

if results:
    for i, res in enumerate(results):
        #print(res.path)
        result_filename = res.path.split("\\")[-1]
        print(result_filename)
        #res.save(filename=result_filename)
        pred_class_index = res.probs.top1  # Get the index of the top prediction
        pred_class_name = res.names[pred_class_index]  # Get the top prediction class
        pred_confidence = res.probs.data[pred_class_index].item()  # Get confidence score
        print(json.dumps({"class": pred_class_name, "confidence": pred_confidence}))

cast_ok_0_16.jpeg
{"class": "OK", "confidence": 0.9287940859794617}


In [ ]:
# Upload the result image to Google Cloud Storage
# Ensure the destination path is correct and exists

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to a Cloud Storage bucket after verifying it exists."""
    #if not os.path.exists(source_file_name):
    #    raise FileNotFoundError(f"Error: Source file '{source_file_name}' not found.")
    #else:
    #    print(f"Source file '{source_file_name}' found.")

    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_filename(source_file_name)
        gs_path = f"gs://{bucket_name}/{destination_blob_name}"
        print(f"Uploaded storage object to {gs_path}")
        return gs_path  # Return the gs:// path

    except Exception as e:
        print(f"Error uploading file: {e}")
        return None  # Return None if an error occurs
    
destination_blob_name = 'result/' + download_file_name
source_file_name = results[0].save_dir + '\\cast_ok_0_16.jpg' # download_file_name
print(source_file_name)
upload_blob(bucket_name, source_file_name, destination_blob_name)

runs\classify\predict36\cast_ok_0_16.jpg
Uploaded storage object to gs://metal_casting_images/result/cast_ok_0_16.jpeg


'gs://metal_casting_images/result/cast_ok_0_16.jpeg'